In [9]:
import polars as pl
import numpy as np
from sklearn.neighbors import KernelDensity
from scipy.optimize import minimize
from scipy.special import gammaln
from utils import donation_data, rfm_data, p1x_data

import altair as alt
from IPython.display import display_markdown

In [13]:
data = donation_data()
rfm_df = rfm_data(data)
rfm_summary = p1x_data(rfm_df)
rfm_summary.collect()

P1X,t_x,Count
i8,i32,u32
6,6,1203
5,6,728
5,5,335
4,6,512
4,5,284
…,…,…
1,4,78
1,3,129
1,2,277


### Parameter Estimation